# VROB1 TP Robin GAIGNOUX

In [1]:
import cv2
import numpy as np
import glob

## Load video

In [2]:
video_path = "baboon.mp4"
cap = cv2.VideoCapture(video_path)
    
ret, I0 = cap.read() 
I0 = cv2.rotate(I0, cv2.ROTATE_90_CLOCKWISE)
resize_factor = 3

def copy_and_resize(I, resize_factor = resize_factor):
    I_copy = np.copy(I)
    I_resized = cv2.resize(I, (I.shape[1] // resize_factor, I.shape[0] // resize_factor))

    return I_resized

# Resize the frame for displaying purpose
I0_resized = copy_and_resize(I0)

## Pick points

⚠️ Pick points in clockwise order : topleft, topright, bottomright, bottomleft ⚠️

In [3]:
pts = []
nb_pts = 4
I0_copy = np.copy(I0_resized)

def select_points(event, x, y, flags, param):
    global pts
    if event == cv2.EVENT_LBUTTONDOWN and len(pts) < nb_pts:
        pts.append((x, y)) # column, row

cv2.namedWindow("Image")
cv2.setMouseCallback("Image", select_points)

while True:
    for pt in pts:
        cv2.circle(I0_copy, pt, 2, (0, 0, 255), -1)
    cv2.imshow("Image", I0_copy)

    key = cv2.waitKey(1) & 0xFF
    if len(pts) == 4:
        break

    if key == 27:
        print("Select 4 points !")


cv2.destroyAllWindows()

In [4]:
# Resize back to get the true values
corners_I0 = np.array([(pt[0] * resize_factor, pt[1] * resize_factor) for pt in pts])
print("Corners in I0 :", corners_I0)

Corners in I0 : [[ 138  264]
 [ 957  273]
 [ 930 1104]
 [ 144 1086]]


## Camera calibration

In [5]:
# Chessboard pattern size
pattern_size = (9, 6)
square_size = 1

# Chessboard points in object coordinate system
object_points = np.zeros((np.prod(pattern_size), 3), dtype=np.float32)
object_points[:, :2] = np.indices(pattern_size).T.reshape(-1, 2)

# Lists to store object points (3D points) and image points (2D projections)
# The points are matched: the 2D point at index i corresponds to the projection of the 3D point at index i
obj_points = []
img_points = []
 
images = glob.glob('calibration_images\\*.jpg')
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001) # Corner refinement criteria
 
for fname in images:
    img = cv2.imread(fname, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
 
    # If found, add object points, image points (after refining them)
    if ret == True:
        obj_points.append(object_points)
 
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        img_points.append(corners2)
 
        # Draw and display the corners
        cv2.drawChessboardCorners(img, pattern_size, corners2, ret)
        resized = cv2.resize(img, (img.shape[1] // resize_factor, img.shape[0] // resize_factor))
        cv2.imshow('img', resized)
        cv2.waitKey(100)
 
cv2.destroyAllWindows()

# Calibrate the camera
ret, K, dist, _, _ = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)
K_inv = np.linalg.inv(K)

# Print intrinsic parameters
print("Intrinsic Matrix :\n", K)
print("Distortion coeffs :\n", dist)

Intrinsic Matrix :
 [[1.32770382e+03 0.00000000e+00 5.32455196e+02]
 [0.00000000e+00 1.32641206e+03 9.58009260e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coeffs :
 [[ 2.06305258e-01 -1.53101813e+00  8.99855256e-04 -1.68850720e-03
   3.94998963e+00]]


## Compute Transformation from World to I0

In [6]:
def to_homogeneous(points):
    ones = np.ones((points.shape[0], 1))
    return np.hstack((points, ones))

def to_cartesian(points):
    return points[:, :-1] / points[:, -1][:, np.newaxis]

def compute_H0(corners_W, corners_I0):
    H_W_to_I0, _ = cv2.findHomography(corners_W, corners_I0)

    """
    # Paper method : "Markerless Tracking using Planar Structures in the Scene" (page 5/9)
    s = np.linalg.norm(H_W_to_I0[:,1]) / np.linalg.norm(H_W_to_I0[:,0])

    D_inv = np.eye(3, dtype=np.float32)
    D_inv[1, 1] = 1/s

    H_W_to_I0 = H_W_to_I0 @ D_inv
    """

    return H_W_to_I0

def compute_T_normalized(H, K_inv):
    R_t = K_inv @ H # [R|t] not normalized
    r1 = R_t[:, 0]
    r2 = R_t[:, 1]
    
    # Normalize the columns to ensure unit vectors
    scale = np.linalg.norm(r1)
    r1 = r1 / scale
    r2 = r2 / scale
    r3 = np.cross(r1, r2)
    
    R = np.column_stack([r1, r2, r3])
    t = R_t[:, 2] / scale
    
    T = K @ np.column_stack([R, t]) # K[R|t] normalized
    
    return T, R, t

def transform_points(points, T):
    return points @ T.T # because : (T @ points.T).T = points @ T.T

def project_points(points_W, T_W_to_I):
    points_I = to_cartesian(transform_points(points_W, T_W_to_I)).astype(np.int32)
    
    return points_I.reshape(-1, 2)

# Real-world coordinates of the 4 corners
corners_W = np.array([[0, 0, 0], [0.185, 0, 0], [0.185, 0.185, 0], [0, 0.185, 0]])

# Compute the transformations
H_W_to_I0 = compute_H0(corners_W, corners_I0)
T_W_to_I0, _, _ = compute_T_normalized(H_W_to_I0, K_inv)

## Show axes (x, y, z) in I0

In [ ]:
# Axes in world coordinate system
axes_W = np.array([[0, 0, 0, 1], [0.185/2, 0, 0, 1], [0, 0.185/2, 0, 1], [0, 0, -0.185/2, 1]])

# Axes in camera coordinate system
axes_I0 = project_points(axes_W, T_W_to_I0)
origin, x_axis, y_axis, z_axis = axes_I0.astype(np.int32)

# Draw axes
I0_copy = np.copy(I0)
cv2.arrowedLine(I0_copy, origin, x_axis, (255, 0, 0), 7)
cv2.arrowedLine(I0_copy, origin, y_axis, (0, 255, 0), 7)
cv2.arrowedLine(I0_copy, origin, z_axis, (0, 0, 255), 7)

I0_resize = cv2.resize(I0_copy, (I0_copy.shape[1] // resize_factor, I0_copy.shape[0] // resize_factor))
cv2.imshow("Axes", I0_resize)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Match keypoints between Ik and Ik+1 to compute transformation from world to Ik+1

In [8]:
def mask_I(I, corners):
    corners = corners.astype(np.int32)
    height, width = I.shape[:2]
    mask = np.zeros((height, width), dtype=np.uint8)
    cv2.fillPoly(mask, [corners], 255)
    masked_I = cv2.bitwise_and(I, I, mask=mask)

    return masked_I

masked_I0 = mask_I(I0, corners_I0)
masked_I0_resize = copy_and_resize(masked_I0)

cv2.imshow("Masked", masked_I0_resize)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
# Output videos
fps = 30
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_result = cv2.VideoWriter("result.mp4", fourcc, fps, (I0.shape[1], I0.shape[0]))
video_sift = cv2.VideoWriter("sift.mp4", fourcc, fps, (I0.shape[1] * 2, I0.shape[0]))

frame_number = 1
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Display frames during the process
display_frames = True

In [10]:
rotations = []
translations = []

if display_frames:
    cv2.namedWindow("Axes")
    cv2.namedWindow("Matches")
    waitkey = 1

# When frame number = 0, the previous frame is I0
Ik = masked_I0
corners = corners_I0
H_I0_to_Ik = np.eye(3)

while True:
    ret, Ik1 = cap.read()
    Ik1 = cv2.rotate(Ik1, cv2.ROTATE_90_CLOCKWISE)

    if not ret:
        break

    print("Current Frame :", frame_number, "/", frame_count)

    # Compute keypoints in Ik and Ik1
    detector = cv2.SIFT_create()
    kpk, desk = detector.detectAndCompute(Ik, None)
    kpk1, desk1 = detector.detectAndCompute(Ik1, None)

    # Match the keypoints
    bf = cv2.BFMatcher(cv2.NORM_L2)
    matches = bf.match(desk, desk1)
    matches = sorted(matches, key=lambda x: x.distance)

    if len(matches) < 4:
        print("Not enough matches for homography")
        continue

    # Points in image plane coordinates
    x_Ik = np.array([kpk[m.queryIdx].pt for m in matches]).reshape(-1, 2)
    x_Ik1 = np.array([kpk1[m.trainIdx].pt for m in matches]).reshape(-1, 2)

    # Compute homography between Ik and Ik1
    H_Ik_to_Ik1, RANSAC_mask = cv2.findHomography(x_Ik, x_Ik1, cv2.RANSAC, 3.0)
    matches_inliers = [matches[i] for i in range(len(matches)) if RANSAC_mask[i] == 1]

    # Compute homography between Ik and Ik1
    H_I0_to_Ik1 = H_Ik_to_Ik1 @ H_I0_to_Ik

    # Compute transformation between world and Ik1
    H_W_to_Ik1 = H_I0_to_Ik1 @ H_W_to_I0
    T_W_to_Ik1, R, t = compute_T_normalized(H_W_to_Ik1, K_inv)
    rotations.append(R)
    translations.append(t)

    # Axes in camera coordinate system
    axes_Ik1 = project_points(axes_W, T_W_to_Ik1)
    origin, x_axis, y_axis, z_axis = axes_Ik1.astype(np.int32)

    # Update corners, Ik and H_I0_to_Ik
    corners = np.float32(corners).reshape(-1, 1, 2)
    corners = cv2.perspectiveTransform(corners, H_Ik_to_Ik1)
    Ik = mask_I(Ik1, corners)
    H_I0_to_Ik = H_I0_to_Ik1

    # Draw matches
    I_matches = cv2.drawMatches(Ik, kpk, Ik1, kpk1, matches_inliers, None, matchColor=(0, 255, 0), singlePointColor=(255, 0, 0), flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    # Draw polylines
    cv2.polylines(Ik1, [corners.astype(np.int32)], isClosed=True, color=(0, 165, 255), thickness=3)

    # Draw axes
    cv2.arrowedLine(Ik1, origin, x_axis, (255, 0, 0), 7)
    cv2.arrowedLine(Ik1, origin, y_axis, (0, 255, 0), 7)
    cv2.arrowedLine(Ik1, origin, z_axis, (0, 0, 255), 7)

    video_result.write(Ik1)
    video_sift.write(I_matches)
    frame_number += 1

    if display_frames:
        # Display matches
        I_matches_resize = cv2.resize(I_matches, (I_matches.shape[1] // resize_factor, I_matches.shape[0] // resize_factor))
        cv2.imshow("Matches", I_matches_resize)

        # Display axes
        Ik1_resize = cv2.resize(Ik1, (Ik1.shape[1] // resize_factor, Ik1.shape[0] // resize_factor))
        cv2.imshow("Axes", Ik1_resize)

        key = cv2.waitKey(waitkey) & 0xFF

        if key == ord('p'):
            waitkey = 1 if waitkey == 0 else 0

        if key == ord('q'):
            break

    
video_sift.release()
video_result.release()

if display_frames:
    cv2.destroyAllWindows()  


Current Frame : 1 / 119
Current Frame : 2 / 119
Current Frame : 3 / 119
Current Frame : 4 / 119
Current Frame : 5 / 119
Current Frame : 6 / 119
Current Frame : 7 / 119
Current Frame : 8 / 119
Current Frame : 9 / 119
Current Frame : 10 / 119
Current Frame : 11 / 119
Current Frame : 12 / 119
Current Frame : 13 / 119
Current Frame : 14 / 119
Current Frame : 15 / 119
Current Frame : 16 / 119
Current Frame : 17 / 119
Current Frame : 18 / 119
Current Frame : 19 / 119
Current Frame : 20 / 119
Current Frame : 21 / 119
Current Frame : 22 / 119
Current Frame : 23 / 119
Current Frame : 24 / 119
Current Frame : 25 / 119
Current Frame : 26 / 119
Current Frame : 27 / 119
Current Frame : 28 / 119
Current Frame : 29 / 119
Current Frame : 30 / 119
Current Frame : 31 / 119
Current Frame : 32 / 119
Current Frame : 33 / 119
Current Frame : 34 / 119
Current Frame : 35 / 119
Current Frame : 36 / 119
Current Frame : 37 / 119
Current Frame : 38 / 119
Current Frame : 39 / 119
Current Frame : 40 / 119
Current F

In [11]:
data = []

for R, t in zip(rotations, translations):
    line = np.hstack((R.flatten(), t))
    data.append(line)

data = np.array(data)
np.savetxt("poses.txt", data)